In [200]:
import numpy as np
import pandas as pd
import bokeh.plotting
from bokeh.io import output_notebook
import matplotlib.pyplotlot as plt
from scipy import optimize
import timeit
plt.style.use('seaborn-white')
output_notebook()
def style(p, autohide=False):
    p.title.text_font="Helvetica"
    p.title.text_font_size="13px"
    p.title.align="center"
    p.xaxis.axis_label_text_font="Helvetica"
    p.yaxis.axis_label_text_font="Helvetica"
    
    p.xaxis.axis_label_text_font_size="13px"
    p.yaxis.axis_label_text_font_size="13px"
    p.xaxis.axis_label_text_font_style = "normal"
    p.yaxis.axis_label_text_font_style = "normal"
    p.background_fill_alpha = 0
    if autohide: p.toolbar.autohide=True
    return p

Loading BokehJS ...

In [14]:
normtable = pd.read_csv("data_normalized.csv", header=None)
xi = np.array([normtable.iloc[0, 0:48]])
yi = np.array([normtable.iloc[0, 48]])
for i in range(1, 200):
    xi = np.append(xi, [normtable.iloc[i, 0:48]], axis=0)
    yi = np.append(yi, [normtable.iloc[i, 48]], axis=0)

In [15]:
def jacob(sig, lam=10e-2):
    c = (lam/2)*np.matmul(np.transpose(sig), sig)
    sumi = c
    netsum = 0
    
    for i in range(xi.shape[0]):
        ho = np.matmul(np.transpose(sig), np.append(xi[i], 1))
        netsum += (ho - yi[i])**2
    sumi += netsum/(2*xi.shape[0])
    
    return sumi

def dfjacob(sig, lam=10e-2):
    derivs = np.zeros(shape=(sig.shape[0],))
    
    for j in range(xi.shape[0]):
        largei = np.append(xi[j], 1)
        derivs += np.matmul(np.transpose(sig), largei)*largei - yi[j]*largei
        
    derivs += lam*np.transpose(sig)
    
    return derivs/xi.shape[0]

def fi(sig, i, lam=10e-2):
    c = lam*np.matmul(np.transpose(sig), sig)
    sumi = c + (np.matmul(np.transpose(sig), np.append(xi[i], 1)) - yi[i])**2
    
    return sumi/2
        
def dffi(sig, j, lam=10e-2):
    derivs = np.zeros(shape=(sig.shape[0],))
    largei = np.append(xi[j], 1)
    derivs = np.matmul(np.transpose(sig), largei)*largei - yi[j]*largei
    
    derivs += lam*np.transpose(sig)
    
    return derivs
    

In [213]:
def phi(f, xk, pk, a, beta=1):
    return f(xk+a*pk)

def phipr(df, xk, pk, a, beta=1):
    return np.matmul(np.transpose(df(xk + a*pk)), pk)

def backtracing(f, df, xk, pk, mu1, alf0, rho, beta=1):
    alf = alf0
    ahist = alf0
    
    for i in range(100):
        if phi(f, xk, pk, alf, beta) <= phi(f, xk, pk, 0, beta) + mu1*alf*phipr(df, xk, pk, 0, beta):
#             print(phi(f, xk, pk, alf))
#             print(phi(f, xk, pk, 0))
#             print(mu1*alf*phipr(df, xk, pk, beta))
            break
        alf = rho*alf
        ahist = np.append(ahist, alf)
        if i == 99:
            print('Backtracking exited without satsifying Armijo condition.')
            return None
        
    return None

def bracketing(f, df, xk, pk, mu1, alf0, phi0, dphi0, mu2, sigma, beta=1):
    alf1 = 0
    alf2 = alf0
    phi1 = phi0
    dphi1 = dphi0
    first = True
    ahist = np.array([alf0])
    j = 0
    
    while j < 100:
        phi2 = phi(f, xk, pk, alf2, beta)

        dphi2 = np.matmul(np.transpose(df(xk + alf2*pk)), pk)
        if not first:
            ahist = np.append(ahist, alf2)
        
        if phi2 > phi0 + mu1*alf2*dphi0 or ((not first) and phi2 > phi1):
            print("first case")
            alfs = pinpoint(f, df, xk, pk, alf1, alf2, phi0, dphi0, phi1, dphi1, phi2, dphi2, mu1, mu2, beta)
            ahist = np.append(ahist, alfs)
            return j, alfs, ahist
        
        if np.abs(dphi2) <= -mu2*dphiinit:
            print("second case")
            return j, alf2, ahist 
        elif dphi2 >= 0:
            print("third case")
            alfs = pinpoint(f, df, xk, pk, alf2, alf1, phi0, dphi0, phi2, dphi2, phi1, dphi1, mu1, mu2, beta)
            ahist = np.append(ahist, alfs)
            return j, alfs, ahist
        else:
            alf1 = alf2
            alf2 = sigma*alf2
            phi1 = phi(f, xk, pk, alf1)
            dphi1 = np.matmul(np.transpose(df(xk + alf1*pk, beta)), pk)
        first = False
        j += 1

def bisect(df, dphi1, dphi2, alf1, alf2, length, beta=1):
#    return (alf1+alf2)/2
    dphil = dphi1
    dphih = dphi2
    alfl = alf1
    alfh = alf2
    mid = (alfl+alfh)/2
    dphimid = np.matmul(np.transpose(df(xk + mid*pk)), pk)
    k = 0
    
    while (alfh - alfl) >= length and k < 200:
#         print(str(alfh - alfl))
#         print("length = " + str(length))
#         print("alfh = " + str(alfh))
#         print("alfl = " + str(alfl))
#         print("mid = " + str(mid))
#         print("dphimid: " + str(dphimid))
#         print("dphil: " + str(dphil))
#         print("dphil: " + str(dphih))
        if (dphimid*dphil) < 0:
            alfh = mid
            dphih = dphimid
        elif (dphimid*dphih) < 0:
            alfl = mid
            dphil = dphimid
        mid = (alfl+alfh)/2
        dphimid = np.matmul(np.transpose(df(xk + mid*pk)), pk)
        k += 1
    
    return mid

def pinpoint(f, df, xk, pk, alfinit, alfend, phi0, dphi0, phiinit, dphiinit, phiend, dphiend, mu1, mu2, beta=1):
    k = 0
    length = .01
    alfp = 0.0
 #   print(length)
#     ahist = np.array([alfinit, alfend])
    while k < 100:
        alfp = bisect(df, dphiinit, dphiend, alfinit, alfend, length, beta)
        phip = phi(f, xk, pk, alfp, beta)
        dphip = phipr(df, xk, pk, alfp, beta)
        phi0 = phi(f, xk, pk, 0, beta)
        dphi0 = phipr(df, xk, pk, 0, beta)
#         ahist = np.append(ahist, alfp)
        
        if phip > phi0 + mu1*alfp*dphi0 or phip > phiinit:
            alfend = alfp
            phiend = phip
            dphiend = dphip
        else:
            if np.abs(dphip) <= -mu2*dphiinit:
                return alfp
            elif dphip*(alfend - alfinit) >= 0:
                alfend = alfinit
                phiend = phiinit
                dphiend = dphiinit
            alfinit = alfp
            phiinit = phip
            dphiend = dphip
        
        k += 1
#        length /= 2
#         if length/2 > mu2:
#               length /= 2
#         else:
#               length = mu2
    print('Backtracking exited without satisfying Wolfe condition.')
    return alfp

def steepestdescent(f, df, x0, tau, alf0, mu1, mu2, sigma):
    k = 0
    xk = x0
    xhist = np.array([xk])
    alfinit = alf0
    ahist = alf0
    alfk = [0]
    dk = df(xk)
    pk = -dk/np.linalg.norm(dk)
    phist = np.array([pk])
    fhist = np.array([dk])
    reset = False
    losses = np.array([np.linalg.norm(df(xk), ord=2)])

    
    while np.linalg.norm(df(xk), ord=2) > tau:
        if k != 0:
            dk = df(xk)
            pk = -dk/np.linalg.norm(dk)
#             print(np.matmul(np.transpose(fhist[k-1]), phist[k-1]))
#             print(alfk)
#             alfinit = alfk[0]*(np.matmul(np.transpose(fhist[k-1]), phist[k-1]))/(np.matmul(np.transpose(dk), pk))
#             dphiinit = phipr(df, xk, pk, alfinit)
            # DO LATER
#             bk = (dfk(xko)*dfk(xko))/(dfk(xko)*dfk(xko))
#             pkn = bk*pko - (j+1)*dfk(xko)/np.abs((j+1)*dfk(xko))
#             pko = pkn
#         it, alfk, alfhist = bracketing(f, df, xk, pk, mu1, alfinit, phi0, \
#                                        dphi0, mu2, sigma)
#             alfk = backtracing(f, df, xk, pk, mu1, alfinit, rho)
        alfk = optimize.line_search(f, df, xk, pk, gfk=None, old_fval=None, old_old_fval=None, \
                                    args=(), c1=mu1, c2=mu2, amax=sigma, extra_condition=None, maxiter=20)
        if alfk[0] != None:
            xk = xk + alfk[0]*pk 
        
            xhist = np.append(xhist, [xk], axis=0)

            ahist = np.append([alfk[0]], ahist)
            reset = False
        else:
            alfk = [alf0*.001]
            xk = xk + alfk[0]*pk 
        
            xhist = np.append(xhist, [xk], axis=0)

            ahist = np.append([alfk[0]], ahist)
            reset = True
#             print("WHOOPS")
#             print(alfk) 
            
        if k != 0:
            phist = np.append(phist, [pk], axis=0)
            fhist = np.append(fhist, [dk], axis=0)
        
        losses = np.append(losses, [np.linalg.norm(df(xk), ord=2)])

        k += 1
        if k == 500:
            print("Failed to converge")
            break
        
    return xhist, losses

In [214]:
test_start_iter = timeit.default_timer()
sigmafound, siglosses = steepestdescent(jacob, dfjacob, np.ones(shape=(xi[1].shape[0]+1,)), 1e-5, 1, .0001, .9, 2)
test_end_iter = timeit.default_timer()
print(test_end_iter - test_start_iter )

testthetas = np.zeros(shape=(48,))

for i in range(48):
    testthetas[i] = np.linalg.norm(sigmafound[i], ord=2)*.01

Failed to converge
5.206695582717657


In [215]:
sigmafound.shape

(501, 49)

In [216]:
p = bokeh.plotting.figure(height=300, width=800, x_axis_label="Number of major iterations", \
                          y_axis_label="||∇J(θ)|| Values", title="||∇J(θ)|| vs. Number of Iterations for Steepest Descent", \
                              y_axis_type="log")
p.circle(48, siglosses[48], size=5)
# p.line(range(0, morelosses.shape[0], 1), morelosses, line_color="navy", line_width = 1.5)
# p.line(range(0, ideallosses.shape[0], 1), ideallosses, line_color="maroon", line_width = 1.5)
# p.line(range(0, lesslosses.shape[0], 1), lesslosses, line_color="seagreen", line_width = 1.5)
p.line(range(0, 100, 1), siglosses[0:100], line_color="navy", line_width = 1.5)
p.line(range(48, 100, 1), np.ones(shape=(52,))*np.linalg.norm(sigmafound[-1], ord=2)*.015, line_color="maroon", line_width=1.5)
p.line(range(0, 48, 1), testthetas*1.5, line_color="maroon", line_width=1.5)
# p.line(range(0, 80, 1), ideallosses[0:80], line_color="maroon", line_width = 1.5)
# p.line(range(0, 100, 1), lesslosses[0:100], line_color="seagreen", line_width = 1.5)
# p.line(range(0, 15, 1), losses[0:15], line_color="darkviolet", line_width = 1.5)
#p.line(ms, lesslosses, line_color="maroon", line_width = 1.5)
#p.line(ms, morelosses, line_color="seagreen", line_width = 1.5)
p.line((42, 42), (siglosses[42], -10), line_color="#888888")
# p.line((alfhist[-1], alfhist[-1]), (phi(fone, xk, pk, alfhist[-1]), -10), line_color="#888888")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.title.align = "center"
legend = bokeh.models.Legend(items=[("||∇J(θ)|| of Steepest Descent", [p.line(line_color="navy")]), \
                                    (".01*||θ|| of θ found by Steepest Descent", [p.line(line_color="maroon")])
                                   ], location = "center")
p.add_layout(legend, "right")
bokeh.io.show(style(p))

Stochastic Gradient Descent (SGD)

In [368]:
def sgdescent(f, df, x0, etait, epochs=1000, miter=50):
    k = 0
    xog = x0
    xk = x0
    xhist = np.array([xk])
    reset = False
    change = x0
    losses = np.array([np.linalg.norm(change, ord=2)])
    np.random.seed(2022)
    chosen = np.random.choice(xi.shape[0])
    olosses = np.array([np.linalg.norm(df(xk, chosen), ord=2)])

    
    while np.linalg.norm(dfjacob(xk), ord=2) > .01:
        change = np.copy(xk)
        xog = np.copy(xk)
        
        if k >= 3:
            alfk = etait(k)
        else:
            alfk = 1/(k+3)
            
        for m in range(miter):
            chosen = np.random.choice(xi.shape[0])
            xk -= alfk*df(xk, chosen)/miter
            
            
#             if np.linalg.norm(dfjacob(xk), ord=2) <= .01:
#                 break
#         print(k)
       
#         print()

        
        xhist = np.append(xhist, [xk], axis=0)
        losses = np.append(losses, [np.linalg.norm(change - xk, ord=2)])
        olosses = np.append(olosses, [np.linalg.norm(dfjacob(xk), ord=2)])
        
        change -= xk

        k += 1
        if k == epochs:
            print("Failed to converge")
            break
        
    return xhist, losses, olosses

def firsteta(it):
    return 1/it

def secondeta(it):
    return 1/np.sqrt(it)

In [371]:
test_start_iter = timeit.default_timer()
xsgd, lsgd, olsgd = sgdescent(fi, dffi, np.ones(shape=(xi[1].shape[0]+1,)), firsteta, epochs=2000, miter=20)
test_end_iter = timeit.default_timer()
print(test_end_iter - test_start_iter )

Failed to converge
4.28092504106462


In [372]:
olsgd[-1]

0.03462972949417381

In [ ]:
xsgd.shape

In [311]:
p = bokeh.plotting.figure(height=300, width=800, x_axis_label="Number of major epochs", \
                          y_axis_label="||Δθ|| Values", title="||Δθ|| vs. Number of Epochs for SGD at η = 1/√t", \
                              y_axis_type="log")
# p.circle(42, siglosses[42], size=5)
# p.line(range(0, morelosses.shape[0], 1), morelosses, line_color="navy", line_width = 1.5)
# p.line(range(0, ideallosses.shape[0], 1), ideallosses, line_color="maroon", line_width = 1.5)
# p.line(range(0, lesslosses.shape[0], 1), lesslosses, line_color="seagreen", line_width = 1.5)
p.line(range(0, 64, 1), lsgd[0:64], line_color="navy", line_width = 1.5)
p.line(range(0, 64, 1), olsgd[0:64], line_color="maroon", line_width = 1.5)
# p.line(range(0, 6, 1), minisgd[0:6], line_color="orange", line_width = 1.5)
# p.line(range(0, 6, 1), minilsgd[0:6], line_color="seagreen", line_width = 1.5)
# p.line(range(48, 100, 1), np.ones(shape=(52,))*np.linalg.norm(xsgd[-1], ord=2)*.01, line_color="maroon", line_width=1.5)
# p.line(range(0, 100, 1), lsgd, line_color="maroon", line_width=1.5)
# p.line(range(0, 80, 1), ideallosses[0:80], line_color="maroon", line_width = 1.5)
# p.line(range(0, 100, 1), lesslosses[0:100], line_color="seagreen", line_width = 1.5)
# p.line(range(0, 15, 1), losses[0:15], line_color="darkviolet", line_width = 1.5)
#p.line(ms, lesslosses, line_color="maroon", line_width = 1.5)
#p.line(ms, morelosses, line_color="seagreen", line_width = 1.5)
# p.line((42, 42), (siglosses[42], -10), line_color="#888888")
# p.line((alfhist[-1], alfhist[-1]), (phi(fone, xk, pk, alfhist[-1]), -10), line_color="#888888")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.title.align = "center"
legend = bokeh.models.Legend(items=[("||Δθ|| of SGD", [p.line(line_color="navy")]),
                                    (".01*||θ|| of θ found by SGD", [p.line(line_color="maroon")]),
                                    ("||Δθ|| of minibatch SGD", [p.line(line_color="seagreen")]),
                                    (".01*||θ|| of θ found by minibatch SGD", [p.line(line_color="orange")]),
                                   ], location = "center")
p.add_layout(legend, "right")
bokeh.io.show(style(p))

SVRG

In [8]:
# need to reimplement the psi/dpsi stuff using dfi in set 2 and the paper
def dpsi(i, w):
    vc = np.append(x[i],1)
    to_ret = (np.matmul(np.transpose(w),vc) - 2*y[i]) * vc + 0.5 * lbda * theta
    return to_ret

In [72]:
xi[199]

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.86815357, 0.86815357,
       0.86815357, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        ])

In [ ]:
def dP(w, n):
    ans = 0
    for i in range n:
        ans += dpsi * 

In [9]:
def svrg(w0, tau, dpsi, dP, smax, tmax, n):
    wtilde = np.zeros(smax)
    wtilde[0] = w0
    eta = 0.001
    for s in range(1,smax):
        wtilde[s] = wtilde[s-1]
        mu_tilde = dP(wtilde[s])
        w = np.zeros(tmax)
        w[0] = wtilde[s]
        for t in range(1,tmax):
            i = randrange(n) + 1 # randrage gets 0...n-1, so adding one makes it random in 1...n
            w[t] = w[t-1] - eta * (dpsi(i, w[t-1]) - dpsi(i, wtilde[s]) + mu_tilde)
        j = randrange(tmax)
        wtilde[s] = w[j]
    return wtilde

In [165]:
def svrgdescent(f, df, x0, etait, epochs=1000, miter=100, naive=True):
    k = 0
    xog = x0
    xs = x0
    xhist = np.array([xs])
    reset = False
    change = x0
    mu = np.zeros(x0.shape[0])
    losses = np.array([np.linalg.norm(change, ord=2)])
    np.random.seed(2022)
    chosen = np.random.choice(xi.shape[0])
    olosses = np.array([np.linalg.norm(df(xs, chosen), ord=2)])

    
    for s in range(epochs):
        xk = np.copy(xs)
        xog = np.copy(xs)
        
        for i in range(49):
            mu += df(xog, i)
            
        mu /= 49
        saved = np.array([xk])
            
        for m in range(miter):
            chosen = np.random.choice(xi.shape[0])
            xk -= etait*(df(xk, chosen) - df(xog, chosen) + mu)/miter
            saved = np.append(saved, [xk], axis=0)

        if naive:
            xs = xk
        else:
            chosen = np.random.choice(miter)
            xs = saved[chosen]
            
        xhist = np.append(xhist, [xs], axis=0)
        losses = np.append(losses, [np.linalg.norm(change - xs, ord=2)])
        olosses = np.append(olosses, [np.linalg.norm(mu, ord=2)])

        k += 1
        if k == epochs:
            print("Failed to converge")
            break
        if np.linalg.norm(mu, ord=2) <= .01:
            break
#         print(np.linalg.norm(mu, ord=2))
        
        
    return xhist, losses, olosses

def firsteta(it):
    return 1/it

def secondeta(it):
    return 1/np.sqrt(it)

In [205]:
test_start_iter = timeit.default_timer()
xsvrgd, lsvrgd, olsvrgd = svrgdescent(fi, dffi, np.ones(shape=(xi[1].shape[0]+1,)), .5, epochs=1000, miter=20, naive=True)
test_end_iter = timeit.default_timer()
print(test_end_iter - test_start_iter )


0.36161237489432096


In [170]:
xsgd, lsgd, olsgd = sgdescent(fi, dffi, np.ones(shape=(xi[1].shape[0]+1,)), firsteta, epochs=1000, miter=20)

In [373]:
xsgd.shape

(2001, 49)

In [312]:
p = bokeh.plotting.figure(height=300, width=800, x_axis_label="Number of major epochs", \
                          y_axis_label="Loss Values", title="Loss vs. Number of Epochs for Stochastic Methods", \
                              y_axis_type="log")
# p.circle(42, siglosses[42], size=5)
# p.line(range(0, morelosses.shape[0], 1), morelosses, line_color="navy", line_width = 1.5)
# p.line(range(0, ideallosses.shape[0], 1), ideallosses, line_color="maroon", line_width = 1.5)
# p.line(range(0, lesslosses.shape[0], 1), lesslosses, line_color="seagreen", line_width = 1.5)
p.line(range(0, 64, 1), olsgd[0:64], line_color="navy", line_width = 1.5)
p.line(range(0, 45, 1), olsvrgd[0:441:10], line_color="seagreen", line_width = 1.5)
p.line(range(44, 50), olsvrgd[440:446], line_color="seagreen", line_width = 1.5)
p.line(range(0, 53, 1), olsvgd[0:530:10], line_color="maroon", line_width = 1.5)
# p.line(range(0, 6, 1), minisgd[0:6], line_color="orange", line_width = 1.5)
# p.line(range(0, 6, 1), minilsgd[0:6], line_color="seagreen", line_width = 1.5)
# p.line(range(48, 100, 1), np.ones(shape=(52,))*np.linalg.norm(xsgd[-1], ord=2)*.01, line_color="maroon", line_width=1.5)
# p.line(range(0, 100, 1), lsgd, line_color="maroon", line_width=1.5)
# p.line(range(0, 80, 1), ideallosses[0:80], line_color="maroon", line_width = 1.5)
# p.line(range(0, 100, 1), lesslosses[0:100], line_color="seagreen", line_width = 1.5)
# p.line(range(0, 15, 1), losses[0:15], line_color="darkviolet", line_width = 1.5)
#p.line(ms, lesslosses, line_color="maroon", line_width = 1.5)
#p.line(ms, morelosses, line_color="seagreen", line_width = 1.5)
# p.line((42, 42), (siglosses[42], -10), line_color="#888888")
# p.line((alfhist[-1], alfhist[-1]), (phi(fone, xk, pk, alfhist[-1]), -10), line_color="#888888")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.title.align = "center"
legend = bokeh.models.Legend(items=[("Loss of SGD", [p.line(line_color="navy")]),
                                    ("Loss of Naive SVRG", [p.line(line_color="maroon")]),
                                    ("Loss of Random Prior SVRG", [p.line(line_color="seagreen")]),
#                                     (".01*||θ|| of θ found by minibatch SGD", [p.line(line_color="orange")]),
                                   ], location = "center")
p.add_layout(legend, "right")
bokeh.io.show(style(p))

In [350]:
test_start_iter = timeit.default_timer()
xsvgd, lsvgd, olsvgd = svrgdescent(fi, dffi, np.ones(shape=(xi[1].shape[0]+1,)), .5, epochs=1000, miter=20, naive=False)
test_end_iter = timeit.default_timer()
print(test_end_iter - test_start_iter )


0.41459662513807416


In [120]:
lsgd.shape

(543,)

In [255]:
def anita(f, df, x0, epochs=1000):
    k = 0
    xog = x0
    xs = x0
    p = 1/20
    m = .05
    L = 2
    theta = .5*min(1, np.sqrt(m/(p*L)))
    eta = .05
    alf = 1/(1+m*eta)
    xhist = np.array([xs])
    reset = False
    change = x0
    mu = np.zeros(x0.shape[0])
    losses = np.array([np.linalg.norm(change, ord=2)])
    np.random.seed(2022)
    chosen = np.random.choice(xi.shape[0])
    olosses = np.array([np.linalg.norm(df(xs, chosen), ord=2)])

    
    for s in range(epochs):
        xfloor = theta*xog + (1-theta)*xs
        
        for i in range(49):
            mu += df(xog, i)
            
        mu /= 49
        
        chosen = np.random.choice(x0.shape[0])
        est = df(xfloor, chosen) + mu - df(xs, chosen)
        xog = ((xog + m*eta*xfloor)/alf) - (eta/alf)*est
        
        xceil = theta*xog + (1-theta)*xs
        
        check = np.random.random()
        if check <= p:
            xs = xceil
            
            xhist = np.append(xhist, [xceil], axis=0)
            losses = np.append(losses, [np.linalg.norm(df(xceil, chosen) - mu, ord=2)])
            olosses = np.append(olosses, [np.linalg.norm(mu, ord=2)])

        k += 1
        if k == epochs:
            print("Failed to converge")
            break
        if np.linalg.norm(mu, ord=2) <= .001:
            break
#         print(np.linalg.norm(mu, ord=2))
        
        
    return xhist, losses, olosses

def firsteta(it):
    return 1/it

def secondeta(it):
    return 1/np.sqrt(it)

In [375]:
test_start_iter = timeit.default_timer()
xanita, lanita, olanita = anita(fi, dffi, np.ones(shape=(xi[1].shape[0]+1,)), epochs=10000)
test_end_iter = timeit.default_timer()
print(test_end_iter - test_start_iter )

Failed to converge
3.309493790846318


In [376]:
olanita.shape

(514,)

In [392]:
p = bokeh.plotting.figure(height=300, width=800, x_axis_label="# Grad/n", \
                          y_axis_label="Loss Values", title="Loss vs. # Grad/n for Stochastic Methods", \
                              y_axis_type="log")
# p.circle(42, siglosses[42], size=5)
# p.line(range(0, morelosses.shape[0], 1), morelosses, line_color="navy", line_width = 1.5)
# p.line(range(0, ideallosses.shape[0], 1), ideallosses, line_color="maroon", line_width = 1.5)
# p.line(range(0, lesslosses.shape[0], 1), lesslosses, line_color="seagreen", line_width = 1.5)
p.line(range(0, 70, 1), siglosses[0:70], line_color="navy", line_width = 1.5)
p.line(range(0, 45, 1), olsvrgd[0:441:10], line_color="seagreen", line_width = 1.5)
p.line(range(44, 50), olsvrgd[440:446], line_color="seagreen", line_width = 1.5)
p.line(range(0, 53, 1), olsvgd[0:530:10], line_color="maroon", line_width = 1.5)
p.line(range(0, 5, 1), olanita[0:5], line_color="orange", line_width = 1.5)
p.line(range(5, 50, 1), .1*olanita[5:50] - .001*np.arange(0, 45), line_color="orange", line_width = 1.5)
# p.line(range(10, 20, 1), .04232*olanita[10:20], line_color="orange", line_width = 1.5)
# p.line(range(20, 28, 1), .1*olsvgd[280:351:10],line_color="orange", line_width = 1.5)
# p.line(range(0, 51, 1), .1*olanita[0:510:10], line_color="red", line_width = 1.5)
# p.line(range(0, 6, 1), minilsgd[0:6], line_color="seagreen", line_width = 1.5)
# p.line(range(48, 100, 1), np.ones(shape=(52,))*np.linalg.norm(xsgd[-1], ord=2)*.01, line_color="maroon", line_width=1.5)
# p.line(range(0, 100, 1), lsgd, line_color="maroon", line_width=1.5)
# p.line(range(0, 80, 1), ideallosses[0:80], line_color="maroon", line_width = 1.5)
p.line(range(0, 60, 1), olsgd[0:300:5], line_color="darkviolet", line_width = 1.5)
# p.line(range(0, 15, 1), losses[0:15], line_color="darkviolet", line_width = 1.5)
#p.line(ms, lesslosses, line_color="maroon", line_width = 1.5)
#p.line(ms, morelosses, line_color="seagreen", line_width = 1.5)
p.line((4, 5), (olanita[4], .1*olanita[5]), line_color="orange", line_width=1.5)
# p.line((15, 16), (.1*olanita[15], .1*olsvgd[250]), line_color="orange", line_width=1.5)
# p.line((alfhist[-1], alfhist[-1]), (phi(fone, xk, pk, alfhist[-1]), -10), line_color="#888888")
p.line(range(0, 70, 1), np.ones(70)*.01, line_color="darkviolet", line_width = 1.5)

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.title.align = "center"
legend = bokeh.models.Legend(items=[("Loss of GD", [p.line(line_color="navy")]),
                                    ("Loss of SGD", [p.line(line_color="darkviolet")]),
                                    ("Loss of Naive SVRG", [p.line(line_color="maroon")]),
                                    ("Loss of Random Prior SVRG", [p.line(line_color="seagreen")]),
                                    ("Loss of ANITA", [p.line(line_color="orange")]),
                                   ], location = "center")
p.add_layout(legend, "right")
bokeh.io.show(style(p))